Análisis del DataFrame obtenido mediante YahooFinance. 
Objetivo: Identificar las variables más propensas a influir en el valor de la acción.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
def graph_prediction(real, prediction):
    real_date = real['Date']
    real_close = real['Close']

    plt.plot(real_date[0:len(prediction)], real_close[0:len(prediction)], color='red', label='Valor real de la acción')
    plt.plot(real_date[0:len(prediction)], prediction, color='blue', label='Predicción de la acción')
    
    # Establecer las etiquetas de las fechas en el eje x
    plt.xticks(rotation=45)
    
    plt.ylim(1.1 * np.min(prediction) / 2, 1.1 * np.max(prediction))
    plt.xlabel('Fecha')
    plt.ylabel("Valor de la Acción")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
ticker = yf.Ticker("AAPL")
info = ticker.info
 
data = yf.download("AAPL", start='2010-07-13', end='2023-10-01')
df = pd.DataFrame(data)
df['SMA_10'] = df['Close'].rolling(window=10).mean()
df.dropna(subset=['SMA_10'], inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.tail()

In [ ]:
# Extraemos los datos de la media movil hasta el 2022-09-31 para el entrenamiento, y del 2022-10-01 en adelante para la validación
train_set = df[df['Date'] <= '2022-09-30']
validate_set = df[df['Date'] >= '2022-10-01']
# Eliminamos las columnas innecesarias
train_set.drop(columns=["Open", "High", "Low", "Adj Close", "Volume", "SMA_10"], inplace=True)
validate_set.drop(columns=["Open", "High", "Low", "Adj Close", "Volume", "SMA_10"], inplace=True)

print(train_set)
print(validate_set)

In [ ]:
# Establecer la columna "Date" como índice
train_set.set_index('Date', inplace=True)
validate_set.set_index('Date', inplace=True)

In [ ]:
# Gráfico para observar los datos de entrenamiento y validación.
train_set["Close"].plot(legend=True)
validate_set["Close"].plot(legend=True)
plt.legend(['Train (2015 - 2022/09)', 'Validate (2022/10)'])
plt.show()

In [ ]:
# Normalizamos los datos de entrenamiento entre 0-1
sc = MinMaxScaler(feature_range=(0,1))
train_set_normal = sc.fit_transform(train_set)

In [ ]:
# La red LSTM tendrá como entrada "time_step" datos consecutivos, y como salida 1 dato (es el resultado de la predicción a partir de esos "time_step" datos). Se conforma de esta manera el set de entrenamiento.
time_step = 60
x_train = []
y_train = []
m = len(train_set_normal)

for i in range(time_step, m):
    # Datos: 0-time_step; 1-time_step+1; 2-time_step+2; etc.
    x_train.append(train_set_normal[i-time_step:i, 0])

    y_train.append(train_set_normal[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
# Defino el modelo de entrada y salida de la red
dim_entry = (x_train.shape[1], 1)
dim_out = 1
na = 50

In [ ]:
# Defino la red secuencial
model = Sequential()
model.add(LSTM(units=na, input_shape=dim_entry, activation="tanh"))
model.add(Dense(units=dim_out))
model.compile(optimizer='rmsprop', loss='mse')
model.fit(x_train, y_train, epochs=50, batch_size=32)

In [ ]:
# Preparo los datos de prueba
x_test = validate_set.values
x_test = sc.transform(x_test)

In [ ]:
X_test = []
for i in range(time_step, len(x_test)):
    X_test.append(x_test[i-time_step:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Realizo la predicción
prediction = model.predict(X_test)
prediction = sc.inverse_transform(prediction)

In [ ]:
graph_prediction(validate_set.reset_index(), prediction)